In [1]:
import os

from exa_py import Exa
from openai import OpenAI
import json

EXA_API_KEY= os.environ.get("EXA_API_KEY")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

exa = Exa(api_key = EXA_API_KEY)
client = OpenAI(api_key = OPENAI_API_KEY)
model = 'gpt-4-turbo'

In [2]:
example_topic = 'The latest news on manufacturers for Digital Print Woven Fabric.'

def generate_report_without_exa(topic):
  content = f"Write a comprehensive and professional three paragraph research report about {topic}. Include citations with source, month, and year."
  completion = client.chat.completions.create(
      model = model,
      messages=[
          {"role": "user", "content": content},
      ],
      temperature = 0
      )
  return completion.choices[0].message.content


report = generate_report_without_exa(example_topic)
report

"As of March 2023, the digital print woven fabric industry continues to evolve, driven by advancements in technology and shifts in consumer preferences towards customization and sustainability. A key player in this sector, Spoonflower Inc., has been at the forefront of integrating eco-friendly practices into digital fabric printing. Spoonflower's approach not only emphasizes the reduction of water and dye use but also focuses on using 100% natural fibers, which are crucial for reducing the environmental footprint. Their recent developments have been geared towards enhancing the color vividness and wash durability of digitally printed fabrics, which remain significant concerns for consumers and manufacturers alike (Smith, J., Textile Magazine, March 2023).\n\nAnother significant development in the digital print woven fabric market is the adoption of advanced digital printing technologies by manufacturers such as Epson and Kornit Digital. These companies have introduced high-speed digita

In [3]:
example_topic = "Inman Mills"

def generate_report_without_exa(topic):
  content = f"Write a comprehensive and professional three paragraph research report about {topic}. Include citations with source, month, and year."
  completion = client.chat.completions.create(
      model = model,
      messages=[
          {"role": "user", "content": content},
      ],
      temperature = 0
      )
  return completion.choices[0].message.content


report = generate_report_without_exa(example_topic)
report

"Inman Mills, an iconic name in the American textile industry, has a rich history that dates back to its establishment in 1901 in Inman, South Carolina. Founded by James A. Chapman, the mill was initially set up to capitalize on the booming textile industry in the Southern United States during the early 20th century. Over the years, Inman Mills has evolved, adapting to changes in technology and market demands. The company has been recognized for its commitment to quality and innovation in the production of yarns and fabrics. It specializes in manufacturing a variety of products, including denim, knits, and specialty woven fabrics, which are used in a range of applications from fashion to industrial materials (Smith, Textile Industry Historical Society, July 2021).\n\nThroughout the 20th century, Inman Mills thrived by continually upgrading its technology and expanding its product line. The introduction of synthetic fibers and the adoption of modern manufacturing processes in the 1960s 

In [2]:
# Exa searches each subtopic
def exa_search_each_subquery(subqueries):
  list_of_query_exa_pairs = []
  for query in subqueries:
    search_response = exa.search_and_contents(
      query,
      num_results=5,
      use_autoprompt=True,
      start_published_date="2023-06-01", # To give us only recent information post-June 2023
      highlights={"num_sentences": 5},
    )
    query_object = {
        'subquery': query,
        'results': search_response.results
    }
    list_of_query_exa_pairs.append(query_object)
  return list_of_query_exa_pairs



In [3]:
# Reformat Exa results into a string
def format_exa_results_for_llm(list_of_query_exa_pairs, content_slice=750):
    formatted_string = ""
    for i in list_of_query_exa_pairs:
      formatted_string += f"[{i['subquery']}]:\n"
      for result in i['results']:
        content = result.text if result.text else " ".join(result.highlights)
        publish_date = result.published_date
        formatted_string += f"URL: {result.url}\nContent: {content}\nPublish Date: {publish_date}\n"
      formatted_string += "\n"

    return formatted_string

#example
#print(format_exa_results_for_llm(example_list_of_query_exa_pairs))

In [4]:
def generate_report_from_exa_results(topic, list_of_query_exa_pairs):
  formatted_exa_content = format_exa_results_for_llm(list_of_query_exa_pairs)
  content = f"Write a comprehensive and professional three paragraph research report about {topic} based on the provided information. Include citations in the text using footnote notation ([citation #]), for example [2]. First provide the report, followed by a single `References` section that only lists the URLs (and their published date) used, in the format [#] <url>. For the published date, only include the month and year. Reset the citations index and ignore the order of citations in the provided information. Here is the information: {formatted_exa_content}."

  completion = client.chat.completions.create(
      model=model,
      messages=[
          {"role": "user", "content": content},
      ],
  )

  return completion.choices[0].message.content

In [5]:
def create_custom_function(num_subqueries):
    properties = {}
    for i in range(1, num_subqueries + 1):
        key = f'subquery_{i}'
        properties[key] = {
            'type': 'string',
            'description': 'Search queries that would be useful for generating a report on my main topic'
        }

    custom_function = {
        'name': 'generate_exa_search_queries',
        'description': 'Generates Exa search queries to investigate the main topic',
        'parameters': {
            'type': 'object',
            'properties': properties
        }
    }

    return [custom_function]

In [6]:
def generate_subqueries_from_topic(topic, num_subqueries=6):
    content =  f"I'm going to give you a topic I want to research. I want you to generate {num_subqueries} interesting, diverse search queries that would be useful for generating a report on my main topic. Here is the main topic: {topic}."
    custom_functions = create_custom_function(num_subqueries)
    completion = client.chat.completions.create(
        model = model,
        messages=[
            {"role": "user", "content": content},
        ],
        temperature=0,
        functions = custom_functions,
        function_call = 'auto',
    )

    json_response = json.loads(completion.choices[0].message.function_call.arguments)
    return list(json_response.values())


In [8]:
topic = 'The latest about Schneider Mills in Taylorsville, NC 28681'

def generate_report(topic):
  subqueries = generate_subqueries_from_topic(topic)
  list_of_query_exa_pairs = exa_search_each_subquery(subqueries)
  report = generate_report_from_exa_results(topic, list_of_query_exa_pairs)
  return report

example_report = generate_report(topic)
print(example_report)

Recent developments at Schneider Mills in Taylorsville, NC, underscore the broader challenges currently faced by the furniture industry. As detailed in multiple news sources, Schneider Mills, particularly operating under the Mitchell Gold + Bob Williams brand, has been forced to initiate a shutdown due to financial strains directly attributed to the inability to secure necessary financing to continue operations. This unfortunate turn of events leads to the employment termination of its dedicated workforce with the last operational day being marked as October 25, 2023 [1][2]. This scenario highlights a significant economic downturn within the sector, largely influenced by current economic hardships.

In response to these developments, the economic impact of Schneider Mills’ cessation on the local community is quite substantial. Given that the furniture sector is one of the primary employers in the region, the abrupt halting of Schneider Mills' operations results in significant job losse

In [12]:
topic = 'The latest about Wavepaths, a company focused on developing generative music to adjoin psychedelic-assisted therapies.'

def generate_report(topic):
  subqueries = generate_subqueries_from_topic(topic)
  list_of_query_exa_pairs = exa_search_each_subquery(subqueries)
  report = generate_report_from_exa_results(topic, list_of_query_exa_pairs)
  return report

example_report = generate_report(topic)
print(example_report)

Wavepaths has demonstrated significant advancements in the integration of generative music within psychedelic-assisted therapies, emphasizing how continuous and variable-length sounds can enhance the therapeutic experience. The development of cutting-edge Deep Generative Models (DGMs) which struggle with expressing continuous sounds, especially those mimicking natural movements like "driving" or the resonance from a bow on a string, is a priority for the company as they strive to make their music platform more expressive and adaptive to therapeutic needs [1]. Additionally, leveraging high-quality audio tokenization techniques, as utilized by the Descript Audio Codec (DAC), has allowed Wavepaths to preserve sound fidelity at lower sampling costs, ultimately enhancing the digital music experience in therapeutic settings [2].

In the sphere of healthcare partnerships, Wavepaths has aligned with entities committed to enhancing health sector operations and experiences, such as Wavemaker 360